#### AFSK Demodulator
## Step 6: HDLC

This is a Pynq portion of the AFSK demodulator project.  We will be using the FPGA overlay that we created in Vivado.

At this point we have created the bitstream for "project_06" and copied the bitstream, TCL wrapper, and hardware hand-off file to the Pynq board.

Let's first verify that we can load the module.

In [5]:
from pynq import Overlay, Xlnk
import pynq.lib.dma

overlay = Overlay('project_06.bit')
dma = overlay.demodulator.demodulator_dma
demod = overlay.demodulator.demod

## Implementing HDLC

Below is the implementation of the AFSK demodulator in Python.  We are now going to remove the HDLC code from our existing project.  We need to make a significant change to the Python code here.  We are no longer writing X items and reading back X items.  We are now feeding in audio data and will occassionally get back a packet.  The packet code will need to be dealt with asynchronously.

Luckily, the Pynq environment fully supports Python's asyncio interface for DMA operations.  When the packet arrive, we will decode the AX.25 data and print it, just like before.

In [6]:
import sys
sys.path.append('../../base')

import numpy as np
from scipy.io.wavfile import read
from AX25 import AX25
import time
import asyncio
import concurrent.futures

block_size = 8192

xlnk = Xlnk()

demod.register_map.done = 0

async def read_packet():
    count = 0
    with xlnk.cma_array(shape=(4096,), dtype=np.uint8) as in_buffer:
        while True:
            dma.recvchannel.transfer(in_buffer)
            await dma.recvchannel.wait_async()
            received = dma.recvchannel._mmio.read(dma.recvchannel._offset + 0x28)
            if received < 2:
                print("Done Reading", received)
                break
            count += 1
            packet = ''.join(str(s, encoding='Latin-1') for s in in_buffer[:received-2])
            try:
                print(count, AX25(packet), received)
                # sys.stdout.write('\r%05d' % count)
            except:
                print("decode error: ", packet)

async def write_audio(data):
    with xlnk.cma_array(shape=(block_size,), dtype=np.int16) as out_buffer:
        for i in range(0, len(data), block_size):
            size = len(data[i:i+block_size])
            if size != block_size: break
            out_buffer[:] = data[i:i+block_size]
            dma.sendchannel.transfer(out_buffer)
            await dma.sendchannel.wait_async()
    
    with xlnk.cma_array(shape=(size,), dtype=np.int16) as out_buffer:
        out_buffer[:] = data[i:i+size]
        dma.sendchannel.transfer(out_buffer)
        await dma.sendchannel.wait_async()

    print("Done writing")
    demod.register_map.done = 1
    
audio_file = read('../../base/TNC_Test_Ver-1.102-26400.wav')
sample_rate = audio_file[0]
audio_data = audio_file[1]
delay = 12 # ~446us

bpf_delay = 70
lpf_delay = 50
filter_delay = bpf_delay + lpf_delay

print("Starting...")
start_time = time.time()

# demodulate the audio data
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(
    read_packet(),
    write_audio(audio_data)
))
stop_time = time.time()
sw_exec_time = stop_time - start_time
print('Total execution time: ',sw_exec_time)

Starting...
1 WA6YLB-8>APRS,N6EX-11:$ULTW00000000----0000----000086A00001----0000000000000000?? 84
2 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!? 77
3 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!? 77
4 WA6YLB>APRX46,WA6YLB-14,W6SCE-5:>081839z wa6ylb@theworks.com? 61
5 KC6HUR-2>S4QVYV,W6SCE-5:'.4&l-/k/]"7q}? 40
6 N6XQY-8>GPSLJ,N6EX-9:$GPRMC,013641.06,A,3348.1607,N,11807.4631,W,34.0,090.5,231105,13.,E*73? 96
7 KC6BLF-12>S4PWYS,N6EX-11:'-U l{(u/]"5\}Lost in the West!? 57
8 K6KMA-2>GPSLK,N6EX-3:$GPRMC,013647,A,3350.076,N,11806.996,W,028.3,180.5,231105,013.5,E*69?? 95
9 AE6GR-14>S4PXYW,N6EX-3:'._|l tv/]"6[}? 40
10 AE6MP>SS5PPQ-4,WIDE2-5:`.](n->>/"4W} 38
11 AE6MP>SS5PPQ-4,N6EX-9:`.](n->>/"4W} 38
12 AE6MP>SS5PPQ-4,N6EX-3:`.](n->>/"4W} 38
13 AE6MP>SS5PPQ-2,N6EX-11:`.](n->>/"4W} 38
14 WA6YLB-8>S6QWSY,WA6YLB-14,N6EX-11:'/`0n>vR/]"56}? 47
15 K6LAR-2>APRS,N6EX-3:$GPGGA,040332,3405.438,N,11801.836,W,1,06,1.1,114.2,M,-31.5,M,,*75?? 93
16

113 WA6YLB-8>APRS,N6EX-11:$ULTW00000000----0000----000086A00001----0000000000000000?? 84
114 KG6SKQ-2>APT311,N6EX-9:/230141z3343.89N/11801.89Wk315/000/A=000013/TT3 72
115 K7ACS-4>APRS,WB6WLV-6,WB6JAR-4,WIDE2-1:=3242.36N\11435.89W? PHG92603/Probe Enabled 82
116 KF6KOI>GPSMV,N6EX-9:$GPRMC,014118,A,3347.6411,N,11805.4960,W,000.0,111.4,231105,013.4,E*69?? 97
117 KE6IYC-4>S4SPTT,K7GIL-2,N6EX-9:'-_jl"?k/]"=m}Out and About.? 61
118 N6EX-8>APNU18,WIDE2-3:!3336.35N111748.63W#PHG5660/W1 for Orange County 73
119 N6EX-8>APNU18,N6EX-3:!3336.35N111748.63W#PHG5660/W1 for Orange County 73
120 KF6DQ-10>S4QUYZ,N6EX-9:'.Kfl ~v/]"6E}TM-D700? 47
121 KF6DQ-10>S4QUYZ,W6SCE-5:'.Kfl ~v/]"6E}TM-D700? 47
122 KF6DQ-10>S4QUYZ,W6SCE-5:'.Kfl ~v/]"6E}TM-D700? 47
123 KC6JTN-2>APT311,WA6YLB-8,N6EX-9:!3418.27N/11648.26W>024/024/A=006213 68
124 KC6JTN-2>APT311,WA6YLB-8,N6EX-4,WB6JAR-4,WIDE2-1:!3418.27N/11648.26W>024/024/A=006213 82
125 KC6JTN-2>APT311,WA6YLB-8,W6SCE-5:!3418.27N/11648.26W>024/024/A=006213 68
126 KC6JTN-2>

231 KC6JTN-2>APT311,WA6YLB-8,W6SCE-5:!3419.88N/11649.85W>338/014/A=005458 68
232 KC6JTN-2>APT311,WA6YLB-8,W6SCE-5:!3419.88N/11649.85W>338/014/A=005458 68
233 N6BOX-4>APT310,WB6JAR-4,WIDE2-3:/230145z3331.36N/11709.71Wk109/020/A=000957 75
234 N6BOX-4>APT310,WB6JAR-4,N6EX-3:/230145z3331.36N/11709.71Wk109/020/A=000957 75
235 KC6JTN-2>APT311,K7GIL-2,N6EX-9:!3419.90N/11649.84W>026/026/A=005445 68
236 KC6JTN-2>APT311,K7GIL-2,N6EX-4,N6EX-9:!3419.90N/11649.84W>026/026/A=005445 75
237 KE6IYC-4>S4SPTS,K7GIL-2,WB6JAR-4,WIDE2-3:'-_jl"?k/]">2}? 54
238 KE6IYC-4>S4SPTS,K7GIL-2,N6EX-9:'-_jl"?k/]">2}? 47
239 KE6IYC-4>S4SPTS,K7GIL-2,WB6JAR-4,N6EX-3:'-_jl"?k/]">2}? 54
240 KE6IYC-4>S4SPTS,K7GIL-2,WB6JAR-4,W6SCE-5:'-_jl"?k/]">2}? 54
241 KE6IYC-4>S4SPTS,K7GIL-2,W6SCE-5:'-_jl"?k/]">2}? 47
242 KA1WCC-2>ST0XTU,N6EX-3:`-R{mRl>/ 34
243 AC6VV-2>S4PYPW,WIDE1-3:`.`:oHv>/]"6\}Reachable via arrl.net? 62
244 KN6DB-12>GPSLK,WIDE2-5:$GPGGA,014551,3348.8478,N,11800.1699,W,2,08,1.1,10.1,M,-32.0,M,,*43?? 94
245 KN6DB-12>GPS

338 KY6F-2>S5PYXU,W6PVG-6,N6EX-11:`.B0l ?v/"Ac} 45
339 N6EX-4>APN383,N6EX-9:!3415.46NL11719.90W#PHG4760/W3 in Crestline/A=005300/sceara@ham-radio.com? 99
340 N6EDL-4>S3SUSV,WB6JAR-4,WIDE-1:`-(?l f>/>"8W}ERNIE IN GMC ENVOY? 65
341 N6XQY-8>GPSLJ,N6EX-9:$GPRMC,015042.06,A,3345.9217,N,11756.2873,W,00.0,000.0,231105,13.,E*7E? 96
342 W2NKM-4>STQRTP,RELAY,WIDE-1:`.'/l"tR/]"83}? 47
343 W2NKM-4>STQRTP,WB6JAR-4,WIDE-1:`.'/l"tR/]"83}? 47
344 N6VNI-12>APRS,WB6JAR-4,WIDE,WIDE-1:!3356.05N/11758.01Wk Geo & Kris LaHabra,CA 81
345 K6KMA-2>GPSLK,N6EX-9:$GPRMC,015100,A,3349.511,N,11806.927,W,000.0,001.4,231105,013.5,E*6D?? 95
346 AE6MP>SS5TSR-4,N6EX-9:`.^:l*!>/"4_} 38
347 KE6RYZ>S3UUXT,RELAY,WIDE-1:`.[Jl!h>/]"4N}? 47
348 KE6RYZ>S3UUXT,RELAY,WIDE-1:`.[Jl!h>/]"4N}? 47
349 N6PHX-8>APT311,WB6JAR-4,WIDE3-5:> Steve's Rig 45
350 KF6YVS-12>APT202,WB6JAR-4,WIDE3-5:!0000.000/00000.000>000/000/kf6yvs, Mike 72
351 WA6YLB>APRX46,WA6YLB-14,N6EX-11:=3617.37N/11908.94W_009/000g000t064r000p000....h00b10176dU2ks.c? 96
352

454 KF6MDF-4>GPS,N6EX-3:$GPRMC,015601,V,3354.0188,N,11805.8904,W,000.0,000.0,301105,013.7,E*7B?? 97
455 N6EX-6>APJI23,N6EX-2,SOCAL1-1:}KC6HUR-13>APVR27,TCPIP,N6EX-3*:;IRLP-4494*230156z3419.32NI11826.52W0440220s110IDLE       121
456 N6VNI-12>APRS,WB6JAR-4,WIDE,WIDE-1:!3356.06N/11758.01Wk Geo & Kris LaHabra,CA 81
457 KG6SKQ-2>APT311,N6EX-9:/230156z3343.89N/11801.89Wk315/000/A=000039/TT3 72
458 KA6IHT-6>ID,WIDE2-5:KA6IHT-3/R RELAY/D ADE2-1/B? 53
459 KA6IHT-6>ID,WB6JAR-4,WIDE2-3:KA6IHT-3/R RELAY/D ADE2-1/B? 60
460 KA6IHT-6>ID,WB6JAR-4,N6EX-3:KA6IHT-3/R RELAY/D ADE2-1/B? 60
461 KA6IHT-6>ID,W6SCE-5:KA6IHT-3/R RELAY/D ADE2-1/B? 53
462 KF6KOI>GPSMV,N6EX-9:$GPRMC,015618,A,3347.6380,N,11805.4948,W,000.0,111.4,231105,013.4,E*6A?? 97
463 W6AMH-6>APS224,W6PVG-6,W6SCE-5:=3503.85N/11756.96WrPHG5260 CALCTY? 67
464 KB6CUS>APK101,N6EX-3::KB6CUR   :O MAMA{1? 45
465 KB6CUS>APK101,N6EX-9::KB6CUR   :O MAMA{1? 45
466 K7ACS>APU25N,WB6WLV-6,WB6JAR-4,WIDE2-1:>230156zDX: WB6WLV-11 32.52.60N 116.24.89W 103.4 mile

563 N6BOX-4>APT310,WB6JAR-4,N6EX-3:/230200z3337.70N/11710.26Wk000/048/A=001535 75
564 N6EX-9>UIDIGI,:UIDIGI 1.8 BETA 6 35
565 W2NKM-4>STQRTP,RELAY,WIDE-1:`.'/l"tR/]"8;}? 47
566 AC6VV-2>S4PXYX,WIDE1-3:`._?l [>/]"6_}? 40
567 AC6VV-2>S4PXYX,N6EX-3:`._?l [>/]"6_}? 40
568 KF6DQ-10>S4QPVZ,W6SCE-5:'.OjlhOv/]"6Z}TM-D700? 47
569 N6QFD-2>GPSLJ,WIDE2-5:$GPRMC,020114,A,3409.7103,N,11804.0209,W,14.6,89.2,231105,13.5,E,A*30?? 96
570 N6QFD-2>GPSLJ,N6EX-3:$GPRMC,020114,A,3409.7103,N,11804.0209,W,14.6,89.2,231105,13.5,E,A*30?? 96
571 KE6RYZ>S3UUXT,RELAY,WIDE-1:`.[Jl!h>/]"4A}? 47
572 N6EX-8>APNU18,WIDE2-3:!3336.35N111748.63W#PHG5660/W1 for Orange County 73
573 N6EX-8>APNU18,N6EX-3:!3336.35N111748.63W#PHG5660/W1 for Orange County 73
574 N6EX-8>APNU18,W6SCE-5:!3336.35N111748.63W#PHG5660/W1 for Orange County 73
575 K7ELH-14>S3QRTX,WB6JAR-4,WIDE1,WIDE2-5:'-+(l!X>/]"5'}? 54
576 K7ELH-14>S3QRTX,WB6JAR-4,WIDE1,N6EX-3:'-+(l!X>/]"5'}? 54
577 W6CSP-4>GPSLK,N6EX-9:$GPGGA,020138,3350.5766,N,11755.8471,W,1,10,0.9,50

676 KB6CYS>APRS,WIDE3-7:!3349.73N/11802.82W_225/000t068p000h68b10162/C KB6CYS FOR WX? 86
677 KB6CYS>APRS,N6EX-3:!3349.73N/11802.82W_225/000t068p000h68b10162/C KB6CYS FOR WX? 86
678 KB6CYS>APRS,N6EX-9:!3349.73N/11802.82W_225/000t068p000h68b10162/C KB6CYS FOR WX? 86
679 KB6CYS>APRS,W6SCE-5:!3349.73N/11802.82W_225/000t068p000h68b10162/C KB6CYS FOR WX? 86
680 KB6CYS>APRS,WIDE3-7:!3349.73N/11802.82W_225/000t068p000h68b10162/C KB6CYS FOR WX? 86
681 KB6CYS>APRS,N6EX-3:!3349.73N/11802.82W_225/000t068p000h68b10162/C KB6CYS FOR WX? 86
682 KD6RSQ>APRS,W5NVH-4,WB6WLV-6,WB6JAR-4,WIDE3-1:$ULTW00000000026E074327AB00119A3E000101930145044200000000?? 105
683 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!? 77
684 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!? 77
685 W6MTR>APU25N,W6PVG-6,W6SCE-5:=3620.40N/11916.65W-w6mtr@arrl.net {UIV32N}? 76
686 KN6DB-12>GPSLK,WIDE2-5:$GPRMC,020651,A,3348.8474,N,11800.1689,W,000.0,274.0,231105,013.4,E*6A?? 97
687

782 KE6RYZ>S3UUXT,RELAY,WIDE-1:`.[Jl!h>/]"4A}? 47
783 KE6RYZ>S3UUXT,WB6JAR-4,WIDE-1:`.[Jl!h>/]"4A}? 47
784 N6QFD-2>GPSLJ,N6EX-3:$GPRMC,021144,A,3409.3693,N,11801.9079,W,9.8,172.7,231105,13.5,E,D*30?? 96
785 N6EDL-4>S3SUSU,WB6JAR-4,WIDE-1:`-(?l f>/>"8N}ERNIE IN GMC ENVOY? 65
786 AD6NH>APRS,N6EX-9:;LA LOAD  *221011z3352.18N\11749.77W? 97 In 10 Minutes 79
787 KF6MDF-4>GPS,N6EX-3:$GPRMC,021201,V,3354.0188,N,11805.8904,W,000.0,000.0,301105,013.7,E*78?? 97
788 N6EX-6>APJI23,N6EX-8,SOCAL1-3:}W6AHM>APRS,TCPIP,N6EX-3*:@230210z3350.28N/11818.85W_274/000g006t066r000P000p000h54b10156v6 123
789 N6EX-6>APJI23,N6EX-8,SOCAL1-3:}W6AHM>APRS,TCPIP,N6EX-3*:@230210z3350.28N/11818.85W_274/000g006t066r000P000p000h54b10156v6 123
790 N6EX-6>APJI23,N6EX-2,SOCAL1-1:}W6AHM>APRS,TCPIP,N6EX-3*:@230210z3350.28N/11818.85W_274/000g006t066r000P000p000h54b10156v6 123
791 N6PHX-8>APT311,N6EX-9:> Steve's Rig 38
792 KF6YVS-12>APT202,WB6JAR-4,WIDE3-5:!0000.000/00000.000>000/000/kf6yvs, Mike 72
793 KG6SKQ-2>APT311,N6EX-9:/23

887 N6HGA-2>APS199,W6PVG-6,W6SCE-4,W6SCE-4,WIDE2-5:@230211z3433.90N/11806.46W_204/003g000t058r000p000P000h44b09260+Davis? 116
888 N6HGA-2>APS199,WA6YLB-8,WIDE2,W6SCE-4,WIDE2-5:@230211z3433.90N/11806.46W_204/003g000t058r000p000P000h44b09260+Davis? 116
889 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!? 77
890 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!? 77
891 K7ELH-14>S3QRTW,WB6JAR-4,WIDE1,WIDE2-5:'-+(l!X>/]"50}? 54
892 K7ELH-14>S3QRTW,WB6JAR-4,WIDE1,N6EX-3:'-+(l!X>/]"50}? 54
893 KG6SKQ-2>APT311,N6EX-9:/230216z3343.34N/11800.95Wk155/012/A=000000/TT3 72
894 N6VNI-12>APRS,WB6JAR-4,WIDE,WIDE-1:!3356.05N/11758.01Wk Geo & Kris LaHabra,CA 81
895 N3DAB-6>APT311,W6PVG-6,WB6JAR-4,WIDE3-3:/230216z3510.16N/11828.47Wk321/054/A=003628 82
896 K7ELH-14>S3QRTW,WB6JAR-4,WIDE1,W6SCE-5:'-+(l!X>/]"50}? 54
897 K7ELH-14>S3QRTW,WB6JAR-4,WIDE1,W6PVG-6,W6SCE-5:'-+(l!X>/]"50}? 61
898 KF6HDJ-6>S2TRXU,RELAY,WB6JAR-4,WIDE2-1:'-*Wl ?k/]"4s}? 54
899 N6EX-6

In [3]:
xlnk.xlnk_reset()

In [ ]:
ctrl = dma.recvchannel._mmio.read(dma.recvchannel._offset)
print(ctrl)
dma.recvchannel._mmio.write(dma.recvchannel._offset, (ctrl | 4) & 0xFFFFFFFE)
print(dma.recvchannel._mmio.read(dma.recvchannel._offset+0x04))
dma.recvchannel.start()
dma.sendchannel.start()

In [ ]:
dma.recvchannel.stop()
dma.recvchannel.start()
with xlnk.cma_array(shape=(4096,), dtype=np.uint8) as in_buffer:
    while running:
        start_time = time.time()
        dma.recvchannel.transfer(in_buffer)
        dma.recvchannel.wait()
        dma.recvchannel.stop()
        dma.recvchannel.start()
        received = dma.recvchannel._mmio.read(dma.recvchannel._offset + 0x28)
        print("Received: ", received)
        packet = ''.join(str(s, encoding='Latin-1') for s in in_buffer[:received])
        print(AX25(packet))
        stop_time = time.time()
        sw_exec_time = stop_time - start_time
        print('FPGA read_packet execution time: ',sw_exec_time)


In [ ]:
demod.register_map.done = 1

In [ ]:
print(demod.register_map)